## remaking panel 3D heatmap to reflect UniProt cross-reference file mapping downloaded for 01_2018 releases
- provides UniProt version ID = stable Ensembl ID mapping
- i could search for the canonical ID only, or use the isoform ID for canonical sequences mapped. 


#### HUMAN_9606_idmapping_2018_08_01download.dat

In [7]:
# packages
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("/Users/mariapalafox/Desktop/Toolbox/")
from all_funx import *

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

# set path
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/HUMAN_9606_uniprot_IDmapping/2018_08_01_IDmapping_download/")

print(os.listdir())

['UKBIDs_2018_3963.csv', 'HUMAN_9606_idmapping_column1.txt', 'HUMAN_9606_idmapping_2018_08_01download.dat', 'Ensembl_PRO_filtered_2018IDmappingUKB.txt', 'HUMAN_9606_idmapping_column1_unique.txt']


In [8]:
# uniprot ID filter file, IDs successfully mapped 
# to all ensembl releases, 3953
ukb3953 = pd.read_csv("UKBID_3953set.csv")
print(ukb3953.shape)
print(ukb3953.head(3))


(3953, 1)
       ID
0  A0AVT1
1  A0FGR8
2  A0JNW5


In [4]:
# read in IDdat file
idat = pd.read_table("HUMAN_9606_idmapping_2018_08_01download.dat", header=None)
idat.columns = ['ID', 'xref', 'xref_ID']
idat

,ID,xref,xref_ID
0,P31946,UniProtKB-ID,1433B_HUMAN
1,P31946,Gene_Name,YWHAB
2,P31946,GI,4507949
3,P31946,GI,377656702
4,P31946,GI,67464628
...,...,...,...
5799598,A0A286YFL1,Ensembl_TRS,ENST00000642079
5799599,A0A286YFL1,Ensembl_PRO,ENSP00000493341
5799600,A0A286YFL1,HGNC,HGNC:27001
5799601,A0A286YFL1,GeneTree,ENSGT00530000062954


In [9]:
# filtering for only IDs mapping to ENSP protein stable ID
xensp = idat.loc[idat['xref'] == 'Ensembl_PRO']
print(xensp.shape)
print(len(xensp.ID.unique()))

(106672, 3)
86659


In [11]:
merge_stableID = pd.merge(xensp,ukb3953,how='inner',on=['ID'])
print(len(merge_stableID.ID.unique()))
# 1489 unique ukb ID in merge

1489


In [19]:
# doesnt have stable version of G6PD merge_stableID.loc[merge_stableID['ID'] == 'P11413']
xensp.loc[xensp['ID'] == 'P11413-1']

,ID,xref,xref_ID
559909,P11413-1,Ensembl_PRO,ENSP00000377194
559911,P11413-1,Ensembl_PRO,ENSP00000483686


In [32]:
print("stable Uniprot IDs NOT found in idat 2018 mapping file : ", 3953-1489)
print("stable Uniprot IDs FOUND in idat 2018 mapping file : ", 1489)
print("found stable id/total 3953 percent : ", (1489/3953)*100)
# number of stable uniprot ID's not in idat file

stable Uniprot IDs NOT found in idat 2018 mapping file :  2464
stable Uniprot IDs FOUND in idat 2018 mapping file :  1489
found stable id/total 3953 percent :  37.66759423222869


In [22]:
print(len(merge_stableID.ID.unique()))
print(len(merge_stableID.xref_ID.unique()))
print(merge_stableID.shape)
# all ensp are unique in file, 1489 unique uniprot IDs

2616
(2616, 3)


In [23]:
merge_stableID.to_csv("idat_enspfiltered_x3953stable_out2616ENSP_1489stableUKBID.csv", index=False)

In [39]:
m = pd.merge(xensp,ukb3953,how='right',on=['ID'])
print(len(m.ID.unique()))
m.to_csv("idat_merge_3953.csv", index=False)


3953


## reason why some of the 3953 IDs are not in the file with isoform count (from fasta) and canonical ID number (from idat mapping):
- the idat file has ukb stable ID's mapping to some ENSG IDs and some ENSP IDs. not all ukbIDs map to stable geneID 
- only looking at ID's from uniprot with multiple isoforms, the singleIsoform.csv file in canonical/ has IDs like 'Q96FJ2' which in the idat file do not have any isoform IDs

In [51]:
3953 - 2491
# original IDs then unique IDs following merge with canonical and isoform number file
# 1462 uniprot IDs are not in file for canonical ID or isoform counts. 

1462

## from canonical number analysis, i am going to make the isoform uniprot IDs, then search for ENSP matched proteins before filtering heatmap distance file


In [57]:
path = "/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/CANONICAL_number/"
file='prism_hist_10628ccds_canonicalIDnumber.csv'
isoID = pd.read_csv(path+file, low_memory=False)
describeMe(isoID)
isoID.columns = ['ID', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18433', 'idatLen_minus_fastaLen',
       'canonical_iso_number']
isoID['isoformID'] = isoID['ID'] + '-' + isoID['canonical_iso_number'].astype(str)

# merging with 3953 ID filter
iso3953 = pd.merge(isoID,ukb3953, how='inner', on=['ID'])
print(len(iso3953.ID.unique()))
# merging with idat file filtered for only xref to ENSP
xensp.columns = ['isoformID', 'xref','xref_ID']
iso3953 = iso3953[['ID','fasta_len','canonical_iso_number', 'isoformID']].copy()
isomerge = pd.merge(xensp,iso3953,how='inner',on=['isoformID'])
print(len(isomerge.ID.unique()))


(10628, 10)
Index(['stable_entry', 'idat_isoformNumber', 'fasta_isoformNumber', 'idat_len',
       'fasta_len', 'labeledSet_3991', 'labeledShared_3979',
       'UKBccds_9606_set_18433', 'idatLen_minus_fastaLen',
       'canonical_iso_number'],
      dtype='object')
stable_entry              0
idat_isoformNumber        0
fasta_isoformNumber       0
idat_len                  0
fasta_len                 0
labeledSet_3991           0
labeledShared_3979        0
UKBccds_9606_set_18433    0
idatLen_minus_fastaLen    0
canonical_iso_number      0
dtype: int64
2491
2423


In [59]:
print(len(isomerge.xref_ID.unique())) # all ENSP IDs are unique in file 3524 IDs and rows
isomerge # 2423 unique Uniprot IDs

3524


,isoformID,xref,xref_ID,ID,fasta_len,canonical_iso_number
0,P31946-1,Ensembl_PRO,ENSP00000300161,P31946,2,1
1,P31946-1,Ensembl_PRO,ENSP00000361930,P31946,2,1
2,P62258-1,Ensembl_PRO,ENSP00000264335,P62258,2,1
3,P62258-1,Ensembl_PRO,ENSP00000487356,P62258,2,1
4,P31947-1,Ensembl_PRO,ENSP00000340989,P31947,2,1
...,...,...,...,...,...,...
3519,O43264-1,Ensembl_PRO,ENSP00000200135,O43264,2,1
3520,O95229-1,Ensembl_PRO,ENSP00000363055,O95229,2,1
3521,O95229-1,Ensembl_PRO,ENSP00000378801,O95229,2,1
3522,Q15942-1,Ensembl_PRO,ENSP00000324422,Q15942,2,1


In [63]:
isomerge.to_csv("merge_idat_w_multi_isoform_ukbcanonicalID_3524ENSP_2423UKBID.csv",index=False)

In [65]:
def addcolumnconditionalDrop(mapList, df, dfcol, newcol):
    # same as above but also drops False value rows
    mendel = []
    for g in df[dfcol]:
        if g in mapList:
            mendel.append("True")
        else:
            mendel.append("False")
    df.loc[:, newcol] = mendel
    checkColumnValues(df, newcol)
    print("dropping false")
    df.drop(df[df[newcol] == "False"].index, inplace=True)
    df.reset_index(inplace=True, drop=True)
    print("df shape post drop: ", df.shape)
    return df

# making list from xref_ID columns in file
ensplist = isomerge.xref_ID.tolist()
ensplist[:5]

['ENSP00000300161',
 'ENSP00000361930',
 'ENSP00000264335',
 'ENSP00000487356',
 'ENSP00000340989']

In [80]:
# file with heat map distance scores for all ENSP
path = "/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/TSV_UNIPROT_xref/DISTANCE_HEAT/"
file = "LASER_dynamic_slope_leven_hamming_shared3887_44305rows.csv"
distance = pd.read_csv(path+file, low_memory=False)

# dropping based on not included in isomerge file
distance = addcolumnconditionalDrop(ensplist, distance, 'ENSP', 'idatENSP')
distance



  idatENSP  Count
0    False  28540
1     True  15765
dropping false
df shape post drop:  (15765, 39)


,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac,release,HGNC_gene,dynamic_slope_LEVENnorm,dynamic_slope_HAMMINGnorm,idatENSP
0,ENSP00000003302.4,ENSP00000003302,ENST00000003302.4,ENST00000003302,ENSG00000048028.7,ENSG00000048028,1077,MTAELQQDDAAGAADGHGSSCQMLLNQLREITGIQDPSFLHEALKA...,ENSG00000048028_ENST00000003302_ENSP00000003302,Q96RU2,4,4,7,True,1077,0,0.0,0.0,0.0,0.0,True,UBP28_HUMAN,3,"{45: 'K', 171: 'C', 733: 'C'}",2,1,3,2,1,0,0,0,1.0,0.0,85,USP28,1,1,True
1,ENSP00000005259.4,ENSP00000005259,ENST00000005259.4,ENST00000005259,ENSG00000075790.6,ENSG00000075790,241,MTLQWAAVATFLYAEIGLILIFCLPFIPPQRWQKIFSFNVWGKIAT...,ENSG00000075790_ENST00000005259_ENSP00000005259,Q9UHQ4,4,4,6,True,241,0,0.0,0.0,0.0,0.0,True,BAP29_HUMAN,1,{96: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,85,BCAP29,1,1,True
2,ENSP00000368412.2,ENSP00000368412,ENST00000379117.2,ENST00000379117,ENSG00000075790.6,ENSG00000075790,241,MTLQWAAVATFLYAEIGLILIFCLPFIPPQRWQKIFSFNVWGKIAT...,ENSG00000075790_ENST00000379117_ENSP00000368412,Q9UHQ4,2,2,6,True,241,0,0.0,0.0,0.0,0.0,True,BAP29_HUMAN,1,{96: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,85,BCAP29,1,1,True
3,ENSP00000275428.4,ENSP00000275428,ENST00000275428.4,ENST00000275428,ENSG00000006625.13,ENSG00000006625,188,MANSGCKDVTGPDEESFLYFAYGSNLLTERIHLRNPSAAFFCVARL...,ENSG00000006625_ENST00000275428_ENSP00000275428,O75223,4,4,13,True,188,0,0.0,0.0,0.0,0.0,True,GGCT_HUMAN,2,"{42: 'C', 148: 'K'}",1,1,2,1,1,0,0,0,1.0,0.0,85,GGCT,1,1,True
4,ENSP00000005386.3,ENSP00000005386,ENST00000005386.3,ENST00000005386,ENSG00000005175.5,ENSG00000005175,665,MTSANKAIELQLQVKQNAEELQDFMRDLENWEKDIKQKDMELRRQN...,ENSG00000005175_ENST00000005386_ENSP00000005386,Q9H6T3,3,3,5,True,665,0,0.0,0.0,0.0,0.0,True,RPAP3_HUMAN,2,"{328: 'K', 519: 'C'}",1,1,2,1,1,0,0,0,1.0,0.0,85,RPAP3,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15760,ENSP00000472116.1,ENSP00000472116,ENST00000601423.5,ENST00000601423,ENSG00000161677.11,ENSG00000161677,188,MSQAPGAQPSPPTVYHERQRLELCAVHALNNVLQQQLFSQEAADEI...,ENSG00000161677_ENST00000601423_ENSP00000472116,Q8TAC2,1,5,11,True,188,0,0.0,0.0,0.0,0.0,True,JOS2_HUMAN,1,{168: 'C'},1,0,1,1,0,0,0,0,1.0,0.0,97,JOSD2,1,1,True
15761,ENSP00000472929.1,ENSP00000472929,ENST00000602142.6,ENST00000602142,ENSG00000141968.8,ENSG00000141968,845,MELWRQCTHWLIQCRVLPPSHRVTWDGAQVCELAQALRDGVLLCQL...,ENSG00000141968_ENST00000602142_ENSP00000472929,P15498,1,6,8,True,845,0,0.0,0.0,0.0,0.0,True,VAV_HUMAN,3,"{83: 'C', 652: 'C', 794: 'C'}",3,0,3,3,0,0,0,0,1.0,0.0,97,VAV1,1,1,True
15762,ENSP00000473172.1,ENSP00000473172,ENST00000595090.6,ENST00000595090,ENSG00000183207.14,ENSG00000183207,463,MATVTATTKVPEIRDVTRIERIGAHSHIRGLGLDDALEPRQASQGM...,ENSG00000183207_ENST00000595090_ENSP00000473172,Q9Y230,1,6,14,True,463,0,0.0,0.0,0.0,0.0,True,RUVB2_HUMAN,4,"{177: 'K', 227: 'C', 279: 'K', 427: 'K'}",1,3,4,1,3,0,0,0,1.0,0.0,97,RUVBL2,1,1,True
15763,ENSP00000474096.1,ENSP00000474096,ENST00000605844.6,ENST00000605844,ENSG00000270647.6,ENSG00000270647,592,MSDSGSYGQSGGEQQSYSTYGNPGSQGYGQASQSYSGYGQTTDSSY...,ENSG00000270647_ENST00000605844_ENSP00000474096,Q92804,1,6,6,True,592,0,0.0,0.0,0.0,0.0,True,RBP56_HUMAN,2,"{265: 'K', 268: 'K'}",0,2,2,0,2,0,0,0,1.0,0.0,97,TAF15,1,1,True


In [81]:
print(15765/5)
print(3524-3153.0)


3153.0
371.0


In [96]:
# dropping based on not included in isomerge file
oneisolist = merge_stableID.ID.tolist()
distance_noisoform = addcolumnconditionalDrop(oneisolist, distance, 'ID', 'idat_single_isoform')
distance_noisoform
distance_noisoform = distance_noisoform.round({"hamming_normalized_dist":2, "levenshtein_normalized_dist":2})
checkColumnValues(distance_noisoform, 'dynamic_slope_LEVENnorm') 
checkColumnValues(distance_noisoform, 'dynamic_slope_HAMMINGnorm') 
distance_noisoform[(distance_noisoform['dynamic_slope_LEVENnorm'] == 2)]
# no interesting proteins to pull from single isoform list for heatmaps

  idat_single_isoform  Count
0                True    275
dropping false
df shape post drop:  (275, 40)
   dynamic_slope_LEVENnorm  Count
0                        1    270
1                        2      5
   dynamic_slope_HAMMINGnorm  Count
0                          1    270
1                          2      5


,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac,release,HGNC_gene,dynamic_slope_LEVENnorm,dynamic_slope_HAMMINGnorm,idatENSP,idat_single_isoform
26,ENSP00000449380.1,ENSP00000449380,ENST00000549126.1,ENST00000549126,ENSG00000235307.5,ENSG00000235307,836,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,ENSG00000235307_ENST00000549126_ENSP00000449380,P25440,1,1,5,False,801,35,207.0,0.25,35.0,0.04,True,BRD2_HUMAN,1,{176: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,85,BRD2,2,2,True,True
80,ENSP00000449380.2,ENSP00000449380,ENST00000549126.4,ENST00000549126,ENSG00000235307.9,ENSG00000235307,801,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,ENSG00000235307_ENST00000549126_ENSP00000449380,P25440,2,4,9,True,801,0,0.0,0.00,0.0,0.00,True,BRD2_HUMAN,1,{176: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,92,BRD2,2,2,True,True
135,ENSP00000449380.2,ENSP00000449380,ENST00000549126.4,ENST00000549126,ENSG00000235307.9,ENSG00000235307,801,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,ENSG00000235307_ENST00000549126_ENSP00000449380,P25440,2,4,9,True,801,0,0.0,0.00,0.0,0.00,True,BRD2_HUMAN,1,{176: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,94,BRD2,2,2,True,True
190,ENSP00000449380.2,ENSP00000449380,ENST00000549126.4,ENST00000549126,ENSG00000235307.9,ENSG00000235307,801,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,ENSG00000235307_ENST00000549126_ENSP00000449380,P25440,2,4,9,True,801,0,0.0,0.00,0.0,0.00,True,BRD2_HUMAN,1,{176: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,96,BRD2,2,2,True,True
245,ENSP00000449380.2,ENSP00000449380,ENST00000549126.4,ENST00000549126,ENSG00000235307.9,ENSG00000235307,801,MLQNVTPHNKLPGEGNAGLLGLGPEAAAPGKRIRKPSLLYEGFESP...,ENSG00000235307_ENST00000549126_ENSP00000449380,P25440,2,4,9,True,801,0,0.0,0.00,0.0,0.00,True,BRD2_HUMAN,1,{176: 'K'},0,1,1,0,1,0,0,0,1.0,0.0,97,BRD2,2,2,True,True


In [82]:
distance = distance.round({"hamming_normalized_dist":2, "levenshtein_normalized_dist":2})

checkColumnValues(distance, 'dynamic_slope_LEVENnorm') 
checkColumnValues(distance, 'dynamic_slope_HAMMINGnorm') 
# filter for rows with dynamic 2 score for both hamming and leven normalized
dyn = distance[(distance['dynamic_slope_LEVENnorm'] == 2) & (distance['dynamic_slope_HAMMINGnorm'] == 2)]
#dyn1 = distance[(distance['dynamic_slope_LEVENnorm'] == 1) & (distance['dynamic_slope_HAMMINGnorm'] == 1)]

checkColumnValues(dyn, 'release')  
#checkColumnValues(dyn1, 'release')         

uniqueCount(dyn, 'HGNC_gene')
#uniqueCount(dyn1, 'HGNC_gene')

#dyn.sort_values(by=['stableID_key','release'], ascending=False, inplace=True)

   dynamic_slope_LEVENnorm  Count
0                        1  15585
1                        2    175
2                        5      5
   dynamic_slope_HAMMINGnorm  Count
0                          1  15585
1                          2    175
2                          5      5
   release  Count
0       97     35
1       96     35
2       94     35
3       92     35
4       85     35
len of col:  175
len of col set:  30



In [83]:
# only 3 decimal place column formatting
dyn.head(3)

,ENSPv,ENSP,ENSTv,ENST,ENSGv,ENSG,Length,proSequence,stableID_key,ID,pro_ver,tx_ver,gen_ver,identical_2UKBseq,Length_UKB,len_ensp_minus_ukb,hamming_distance,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,key_8861,entryName,labeled_pos_count,pos_dict,count_C_targets,count_K_targets,found_count,found_count_C,found_count_K,missed_count,missed_count_C,missed_count_K,correct_frac,missed_frac,release,HGNC_gene,dynamic_slope_LEVENnorm,dynamic_slope_HAMMINGnorm,idatENSP
40,ENSP00000340824.1,ENSP00000340824,ENST00000341524.1,ENST00000341524,ENSG00000069424.10,ENSG00000069424,367,MYPESTTGSPARLSLRQTGSPGMIYSTRYGSPKRQLQFYRNLGKSG...,ENSG00000069424_ENST00000341524_ENSP00000340824,Q13303,1,1,10,True,367,0,0.0,0.00,0.0,0.00,True,KCAB2_HUMAN,2,"{124: 'K', 248: 'C'}",1,1,2,1,1,0,0,0,1.0,0.0,85,KCNAB2,2,2,True
128,ENSP00000216780.4,ENSP00000216780,ENST00000216780.4,ENST00000216780,ENSG00000100889.7,ENSG00000100889,640,MAALYRPGLRLNWHGLSPLGWPSCRSIQTLRVLSGDLGQLPTGIRD...,ENSG00000100889_ENST00000216780_ENSP00000216780,Q16822,4,4,7,False,640,0,1.0,0.00,1.0,0.00,True,PCKGM_HUMAN,4,"{92: 'C', 210: 'C', 306: 'C', 431: 'C'}",4,0,4,4,0,0,0,0,1.0,0.0,85,PCK2,2,2,True
134,ENSP00000383076.1,ENSP00000383076,ENST00000400217.2,ENST00000400217,ENSG00000101266.12,ENSG00000101266,255,MYEILKALDYCHSMGIMHRDVKPHNVMIDHEHRKLRLIDWGLAEFY...,ENSG00000101266_ENST00000400217_ENSP00000383076,P68400,1,2,12,False,391,-136,375.0,0.96,136.0,0.35,True,CSK21_HUMAN,3,"{102: 'K', 122: 'K', 329: 'K'}",0,3,0,0,0,3,0,3,0.0,1.0,85,CSNK2A1,2,2,True


## Distance to uniprot from all 5 ensembl releases, 
1. filtered for only rows with 'dynamic_slope_HammingORleven_normalized' score of 2 (equivalent to 2 unique distance scores between sequences scores from stableID_key matched sequences)
2. confirmed all stableID_keys had 5 sequences linked
3. calculating RANGE, AVERAGE, and LENGTH of groupby() stableIDkey file, irrespective of release order

[coppied from L19_12_18_HEAT_DISTANCE..markdown]

In [84]:
# FROM MAPLIB.py
# functions that work on list
def setme(s):
    ls = list(s)
    finalset = set(ls)
    return finalset

def avgme(s):
    return mean(s)

def rangefinder(s):
    maxx = max(s)
    minn = min(s)
    diff = maxx - minn
    return diff

def group_scores(df, colname):
    # hamming_normalized_dist
    # levenshtein_normalized_dist
    lsdf = df.groupby('stableID_key')[colname].apply(list)
    lsdf = pd.DataFrame(lsdf)
        # concat all frames together
        #set_columns = pd.concat([lsdf, avgdf], axis=1)
        # problem with duplicated colnames from concat files
    lsdf['avg_distscr'] = lsdf[colname].apply(avgme)
    lsdf['range_distscr'] = lsdf[colname].apply(rangefinder)
    lsdf['max_dist_scr'] = lsdf[colname].apply(lambda x: max(x))
    print("group df shape: ", lsdf.shape)
    print(lsdf.head(1))
    return lsdf

In [86]:
# creating groupby df's for each score 
hamming_group = group_scores(dyn, 'hamming_normalized_dist')
hamming_group.columns = ['hamming_normalized_dist', 'HAM_avg_distscr', 'HAM_range_distscr',
       'HAM_max_dist_scr']

leven_group = group_scores(dyn, 'levenshtein_normalized_dist')
leven_group.columns = ['levenshtein_normalized_dist', 'LEVEN_avg_distscr', 'LEVEN_range_distscr',
       'LEVEN_max_dist_scr']

# combining 2 score df's
mergescores = pd.concat([hamming_group, leven_group], axis=1)
mergescores.reset_index(inplace=True)

mergescores

group df shape:  (35, 4)
                                                   hamming_normalized_dist  \
stableID_key                                                                 
ENSG00000066933_ENST00000444904_ENSP00000398250  [0.8, 0.0, 0.0, 0.0, 0.0]   

                                                 avg_distscr  range_distscr  \
stableID_key                                                                  
ENSG00000066933_ENST00000444904_ENSP00000398250         0.16            0.8   

                                                 max_dist_scr  
stableID_key                                                   
ENSG00000066933_ENST00000444904_ENSP00000398250           0.8  
group df shape:  (35, 4)
                                                levenshtein_normalized_dist  \
stableID_key                                                                  
ENSG00000066933_ENST00000444904_ENSP00000398250  [0.01, 0.0, 0.0, 0.0, 0.0]   

                                               

,stableID_key,hamming_normalized_dist,HAM_avg_distscr,HAM_range_distscr,HAM_max_dist_scr,levenshtein_normalized_dist,LEVEN_avg_distscr,LEVEN_range_distscr,LEVEN_max_dist_scr
0,ENSG00000066933_ENST00000444904_ENSP00000398250,"[0.8, 0.0, 0.0, 0.0, 0.0]",0.160,0.80,0.80,"[0.01, 0.0, 0.0, 0.0, 0.0]",0.002,0.01,0.01
1,ENSG00000069424_ENST00000341524_ENSP00000340824,"[0.0, 0.0, 0.0, 0.0, 0.52]",0.104,0.52,0.52,"[0.0, 0.0, 0.0, 0.0, 0.04]",0.008,0.04,0.04
2,ENSG00000070182_ENST00000389720_ENSP00000374370,"[0.03, 0.0, 0.0, 0.0, 0.0]",0.006,0.03,0.03,"[0.02, 0.0, 0.0, 0.0, 0.0]",0.004,0.02,0.02
3,ENSG00000070814_ENST00000377797_ENSP00000367028,"[0.18, 0.0, 0.0, 0.0, 0.0]",0.036,0.18,0.18,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00
4,ENSG00000075856_ENST00000228284_ENSP00000228284,"[0.0, 0.0, 0.0, 0.7, 0.7]",0.280,0.70,0.70,"[0.0, 0.0, 0.0, 0.02, 0.02]",0.008,0.02,0.02
5,ENSG00000078808_ENST00000360001_ENSP00000353094,"[0.0, 0.0, 0.0, 0.0, 0.93]",0.186,0.93,0.93,"[0.0, 0.0, 0.0, 0.0, 0.02]",0.004,0.02,0.02
6,ENSG00000082269_ENST00000370479_ENSP00000359510,"[0.92, 0.0, 0.0, 0.0, 0.0]",0.184,0.92,0.92,"[0.17, 0.0, 0.0, 0.0, 0.0]",0.034,0.17,0.17
7,ENSG00000093010_ENST00000407537_ENSP00000384654,"[0.94, 0.0, 0.0, 0.0, 0.0]",0.188,0.94,0.94,"[0.18, 0.0, 0.0, 0.0, 0.0]",0.036,0.18,0.18
8,ENSG00000100889_ENST00000216780_ENSP00000216780,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00
9,ENSG00000101266_ENST00000400217_ENSP00000383076,"[0.96, 0.0, 0.0, 0.0, 0.0]",0.192,0.96,0.96,"[0.35, 0.0, 0.0, 0.0, 0.0]",0.070,0.35,0.35


In [87]:
def map_column_values(refdf, adddf, dickey, dicvalue):
    # make sure refdf has dickey and dicvalue column
    # adddf must have dickey column
    newcol = dicvalue
    refdic = dict(zip(refdf[dickey], refdf[dicvalue]))
    adddf[newcol] = adddf[dickey]
    adddf[dicvalue] = adddf[dicvalue].map(refdic)
    return adddf
# adding genes and uniprot ID
mergescores2 = map_column_values(dyn, mergescores, 'stableID_key', 'HGNC_gene')
mergescores2 = map_column_values(dyn, mergescores2, 'stableID_key', 'ID')
mergescores2

,stableID_key,hamming_normalized_dist,HAM_avg_distscr,HAM_range_distscr,HAM_max_dist_scr,levenshtein_normalized_dist,LEVEN_avg_distscr,LEVEN_range_distscr,LEVEN_max_dist_scr,HGNC_gene,ID
0,ENSG00000066933_ENST00000444904_ENSP00000398250,"[0.8, 0.0, 0.0, 0.0, 0.0]",0.160,0.80,0.80,"[0.01, 0.0, 0.0, 0.0, 0.0]",0.002,0.01,0.01,MYO9A,B2RTY4
1,ENSG00000069424_ENST00000341524_ENSP00000340824,"[0.0, 0.0, 0.0, 0.0, 0.52]",0.104,0.52,0.52,"[0.0, 0.0, 0.0, 0.0, 0.04]",0.008,0.04,0.04,KCNAB2,Q13303
2,ENSG00000070182_ENST00000389720_ENSP00000374370,"[0.03, 0.0, 0.0, 0.0, 0.0]",0.006,0.03,0.03,"[0.02, 0.0, 0.0, 0.0, 0.0]",0.004,0.02,0.02,SPTB,P11277
3,ENSG00000070814_ENST00000377797_ENSP00000367028,"[0.18, 0.0, 0.0, 0.0, 0.0]",0.036,0.18,0.18,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00,TCOF1,Q13428
4,ENSG00000075856_ENST00000228284_ENSP00000228284,"[0.0, 0.0, 0.0, 0.7, 0.7]",0.280,0.70,0.70,"[0.0, 0.0, 0.0, 0.02, 0.02]",0.008,0.02,0.02,SART3,Q15020
5,ENSG00000078808_ENST00000360001_ENSP00000353094,"[0.0, 0.0, 0.0, 0.0, 0.93]",0.186,0.93,0.93,"[0.0, 0.0, 0.0, 0.0, 0.02]",0.004,0.02,0.02,SDF4,Q9BRK5
6,ENSG00000082269_ENST00000370479_ENSP00000359510,"[0.92, 0.0, 0.0, 0.0, 0.0]",0.184,0.92,0.92,"[0.17, 0.0, 0.0, 0.0, 0.0]",0.034,0.17,0.17,FAM135A,Q9P2D6
7,ENSG00000093010_ENST00000407537_ENSP00000384654,"[0.94, 0.0, 0.0, 0.0, 0.0]",0.188,0.94,0.94,"[0.18, 0.0, 0.0, 0.0, 0.0]",0.036,0.18,0.18,COMT,P21964
8,ENSG00000100889_ENST00000216780_ENSP00000216780,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000,0.00,0.00,PCK2,Q16822
9,ENSG00000101266_ENST00000400217_ENSP00000383076,"[0.96, 0.0, 0.0, 0.0, 0.0]",0.192,0.96,0.96,"[0.35, 0.0, 0.0, 0.0, 0.0]",0.070,0.35,0.35,CSNK2A1,P68400


In [88]:
# saving DYNAMIC 2
countss = mergescores2['HGNC_gene'].value_counts()
countss = pd.DataFrame(countss)
# gene counts saved
countss.to_csv("GROUPBY_VALUECOUNTS_distancescores_35stableIDs_dynamic2_idatmapped.csv")
# merge file saved
mergescores2.to_csv("GROUPBY_isoformIDmatched_idat_distancescores_35stableIDs_dynamic2.csv")

# MANUALLY went through merged distance scores filtered dynamic 2, deleted rows with HAM average distance less than 0.1

- final IDs for HEATMAPS: len 93

In [90]:
stableID_key=["ENSG00000167460_ENST00000300933_ENSP00000300933","ENSG00000227345_ENST00000402038_ENSP00000384408","ENSG00000152492_ENST00000392456_ENSP00000376250","ENSG00000101266_ENST00000400217_ENSP00000383076","ENSG00000155100_ENST00000404789_ENSP00000384190","ENSG00000204842_ENST00000550104_ENSP00000446576","ENSG00000184047_ENST00000464942_ENSP00000442360","ENSG00000093010_ENST00000407537_ENSP00000384654","ENSG00000082269_ENST00000370479_ENSP00000359510","ENSG00000185420_ENST00000490107_ENSP00000419184","ENSG00000162923_ENST00000414423_ENSP00000408108","ENSG00000113522_ENST00000378823_ENSP00000368100","ENSG00000153113_ENST00000395813_ENSP00000379158","ENSG00000112118_ENST00000229854_ENSP00000229854","ENSG00000257923_ENST00000546411_ENSP00000450125"]
print(len(stableID_key))

15


In [92]:
heat15 = addcolumnconditionalDrop(stableID_key, dyn, "stableID_key", "top15_dynamic")

# isolate other releases
v85 = heat15[heat15['release'] == 85].copy()
v92 = heat15[heat15['release'] == 92].copy()
v94 = heat15[heat15['release'] == 94].copy()
v96 = heat15[heat15['release'] == 96].copy()
v97 = heat15[heat15['release'] == 97].copy()

v85 = v85[['stableID_key','ID', 'hamming_normalized_dist', 'levenshtein_distance',
           'levenshtein_normalized_dist', 'labeled_pos_count', 'missed_frac','release', 'HGNC_gene']].copy()

v92 = v92[['stableID_key','ID', 'hamming_normalized_dist', 'levenshtein_distance',
           'levenshtein_normalized_dist', 'labeled_pos_count', 'missed_frac','release', 'HGNC_gene']].copy()

v94 = v94[['stableID_key','ID', 'hamming_normalized_dist', 'levenshtein_distance',
           'levenshtein_normalized_dist', 'labeled_pos_count', 'missed_frac','release', 'HGNC_gene']].copy()

v96 = v96[['stableID_key','ID', 'hamming_normalized_dist', 'levenshtein_distance',
           'levenshtein_normalized_dist', 'labeled_pos_count', 'missed_frac','release', 'HGNC_gene']].copy()

v97 = v97[['stableID_key','ID', 'hamming_normalized_dist', 'levenshtein_distance',
           'levenshtein_normalized_dist', 'labeled_pos_count', 'missed_frac','release', 'HGNC_gene']].copy()
v85

dfs = [v85, v92, v94, v96, v97]
for i in dfs:
    i.sort_values(by=['stableID_key'], inplace=True)
    i.set_index(['stableID_key'], inplace=True)
    
    
colscores = pd.concat(dfs, axis=1)  
colscores

  top15_dynamic  Count
0          True     75
dropping false
df shape post drop:  (75, 40)


,ID,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,labeled_pos_count,missed_frac,release,HGNC_gene,ID,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,labeled_pos_count,missed_frac,release,HGNC_gene,ID,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,labeled_pos_count,missed_frac,release,HGNC_gene,ID,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,labeled_pos_count,missed_frac,release,HGNC_gene,ID,hamming_normalized_dist,levenshtein_distance,levenshtein_normalized_dist,labeled_pos_count,missed_frac,release,HGNC_gene
stableID_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENSG00000082269_ENST00000370479_ENSP00000359510,Q9P2D6,0.92,265.0,0.17,1,1.00,85,FAM135A,Q9P2D6,0.0,0.0,0.0,1,0.0,92,FAM135A,Q9P2D6,0.0,0.0,0.0,1,0.0,94,FAM135A,Q9P2D6,0.00,0.0,0.00,1,0.0,96,FAM135A,Q9P2D6,0.00,0.0,0.00,1,0.0,97,FAM135A
ENSG00000093010_ENST00000407537_ENSP00000384654,P21964,0.94,50.0,0.18,5,1.00,85,COMT,P21964,0.0,0.0,0.0,5,0.0,92,COMT,P21964,0.0,0.0,0.0,5,0.0,94,COMT,P21964,0.00,0.0,0.00,5,0.0,96,COMT,P21964,0.00,0.0,0.00,5,0.0,97,COMT
ENSG00000101266_ENST00000400217_ENSP00000383076,P68400,0.96,136.0,0.35,3,1.00,85,CSNK2A1,P68400,0.0,0.0,0.0,3,0.0,92,CSNK2A1,P68400,0.0,0.0,0.0,3,0.0,94,CSNK2A1,P68400,0.00,0.0,0.00,3,0.0,96,CSNK2A1,P68400,0.00,0.0,0.00,3,0.0,97,CSNK2A1
ENSG00000112118_ENST00000229854_ENSP00000229854,P25205,0.00,0.0,0.00,16,0.00,85,MCM3,P25205,0.0,0.0,0.0,16,0.0,92,MCM3,P25205,0.0,0.0,0.0,16,0.0,94,MCM3,P25205,0.93,65.0,0.08,16,1.0,96,MCM3,P25205,0.93,65.0,0.08,16,1.0,97,MCM3
ENSG00000113522_ENST00000378823_ENSP00000368100,Q92878,0.95,139.0,0.11,10,1.00,85,RAD50,Q92878,0.0,0.0,0.0,10,0.0,92,RAD50,Q92878,0.0,0.0,0.0,10,0.0,94,RAD50,Q92878,0.00,0.0,0.00,10,0.0,96,RAD50,Q92878,0.00,0.0,0.00,10,0.0,97,RAD50
ENSG00000152492_ENST00000392456_ENSP00000376250,Q8IVM0,0.67,176.0,0.37,4,0.25,85,CCDC50,Q8IVM0,0.0,0.0,0.0,4,0.0,92,CCDC50,Q8IVM0,0.0,0.0,0.0,4,0.0,94,CCDC50,Q8IVM0,0.00,0.0,0.00,4,0.0,96,CCDC50,Q8IVM0,0.00,0.0,0.00,4,0.0,97,CCDC50
ENSG00000153113_ENST00000395813_ENSP00000379158,P20810,0.93,83.0,0.10,11,1.00,85,CAST,P20810,0.0,0.0,0.0,11,0.0,92,CAST,P20810,0.0,0.0,0.0,11,0.0,94,CAST,P20810,0.00,0.0,0.00,11,0.0,96,CAST,P20810,0.00,0.0,0.00,11,0.0,97,CAST
ENSG00000155100_ENST00000404789_ENSP00000384190,Q8N6M0,0.94,101.0,0.34,2,1.00,85,OTUD6B,Q8N6M0,0.0,0.0,0.0,2,0.0,92,OTUD6B,Q8N6M0,0.0,0.0,0.0,2,0.0,94,OTUD6B,Q8N6M0,0.00,0.0,0.00,2,0.0,96,OTUD6B,Q8N6M0,0.00,0.0,0.00,2,0.0,97,OTUD6B
ENSG00000162923_ENST00000414423_ENSP00000408108,Q9H7D7,0.00,0.0,0.00,1,0.00,85,WDR26,Q9H7D7,0.0,0.0,0.0,1,0.0,92,WDR26,Q9H7D7,0.0,0.0,0.0,1,0.0,94,WDR26,Q9H7D7,0.93,100.0,0.13,1,1.0,96,WDR26,Q9H7D7,0.93,100.0,0.13,1,1.0,97,WDR26


In [93]:
colscores.to_csv("COLUMN_TOP15_idat_isoform_mapped_heatmap_panel3.csv")